In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10767, 1000)

(10767, 1)

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Step 2 Identify Features

In [5]:
from sklearn.feature_selection import SelectKBest, \
                                      SelectFromModel, \
                                      RFE, SelectPercentile, \
                                      SelectFdr, SelectFpr
from sklearn.feature_selection import f_regression, f_classif


### Pipeline: Select K best - PCA - SelectFromModel(RFC) - Logistic Regression

In [6]:
pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_classif)),
    ('pca',PCA(random_state=42)),
    ('sfmlr', SelectFromModel(LogisticRegression(C=0.1, penalty='l1',random_state=42))),
    ('sfmrf', SelectFromModel(RandomForestClassifier(random_state=42,n_jobs=-1))),
    ('lr', LogisticRegression(C=1E10,n_jobs=-1,random_state=42))
])

In [7]:
params = {
    'skb__k':[200,300,500],
    'pca__n_components':[50,100,150],
    'sfmrf__estimator':[  RandomForestClassifier(n_estimators=10, criterion='entropy',random_state=42,n_jobs=-1),
                          RandomForestClassifier(n_estimators=20, criterion='entropy',random_state=42,n_jobs=-1),  
                          RandomForestClassifier(n_estimators=30, criterion='entropy',random_state=42,n_jobs=-1),
                          RandomForestClassifier(n_estimators=40, criterion='entropy',random_state=42,n_jobs=-1),  
                          RandomForestClassifier(n_estimators=50, criterion='entropy',random_state=42,n_jobs=-1),
                        ],                    
}

In [8]:
gspipe = GridSearchCV(pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42))

In [9]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7fa1f046e7b8>)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('sfmlr', SelectFromModel(estimator=LogisticRegression(C=0.1, class_weight=None, dual=...alty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'sfmrf__estimator': [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0....      verbose=0, warm_start=False)], 'skb__k': [200, 300, 500], 'pca__n_components': [50, 100, 

In [10]:
gspipe.best_score_

0.53181818181818186

In [11]:
gspipe.best_estimator_.named_steps

{'lr': LogisticRegression(C=10000000000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=42,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
 'pca': PCA(copy=True, iterated_power='auto', n_components=100, random_state=42,
   svd_solver='auto', tol=0.0, whiten=False),
 'sfmlr': SelectFromModel(estimator=LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=42, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
         prefit=False, threshold=None),
 'sfmrf': SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,


In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T.head(6)

,20,41,23,17,32,38,22,28,2,25,...,24,15,42,33,44,21,18,36,27,13
mean_fit_time,0.08265,0.122,0.1053,0.06128,0.06467,0.1595,0.08798,0.1176,0.06161,0.1017,...,0.1196,0.04083,0.1054,0.0592,0.1411,0.07475,0.05999,0.07255,0.1048,0.1156
mean_score_time,0.002278,0.003456,0.002952,0.001634,0.002088,0.006167,0.002855,0.003662,0.001652,0.003188,...,0.004483,0.001415,0.003491,0.001941,0.004218,0.002429,0.00219,0.002549,0.003693,0.003774
mean_test_score,0.5318,0.5182,0.5136,0.5136,0.5091,0.5,0.4909,0.4909,0.4864,0.4864,...,0.4545,0.4545,0.45,0.4455,0.4455,0.4409,0.4409,0.4409,0.4409,0.4136
mean_train_score,0.8551,0.852,0.8485,0.8389,0.8424,0.851,0.8354,0.8434,0.8051,0.8379,...,0.8525,0.8505,0.8495,0.8545,0.848,0.8556,0.853,0.8535,0.852,0.8535
param_pca__n_components,100,150,100,100,150,150,100,100,50,100,...,100,100,150,150,150,100,100,150,100,50
param_sfmrf__estimator,(),(),(),(),(),(),(),(),(),(),...,(),(),(),(),(),(),(),(),(),()


In [13]:
gspipe_results['mean_fit_time'].sum()

4.054180073738098

In [14]:
#gspipe_results.info(memory_usage='deep')

In [15]:
print("Fit time .1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .1% data: ",gspipe.best_score_)
### Pipeline3 for .1% (220 rows)
picked_pipeline_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_pt1_percent')

Fit time .1% data:  4.05418007374
Best Score .1% data:  0.531818181818


In [16]:
### Pipeline3 for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data: ",gspipe.best_score_)
gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline3_1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_pt5_percent')

Fit time .5% data:  26.4366604328
Best Score .5% data:  0.538181818182


In [17]:
### Pipeline3 for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data: ",gspipe.best_score_)
gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_Pipeline2_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_1_percent')

Fit time 1% data:  53.6561871052
Best Score 1% data:  0.511818181818


In [18]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/Pipeline3_lr_5_percent')

Fit time 5% data:  344.360115528
Best Score 5% data: 0.520334261838
